# Function Calling with Google DeepMind Gemini 2.0 Flash

Function calling is the capability to connect LLMs to external tools and to interact with your code and APIs in a structured way. Instead of generating text responses, LLMs understand when to call specific functions and provide the necessary parameters to execute real-world actions.

Throughout this guide, we'll build a practical weather-based outfit assistant that helps users decide what to wear based on current weather conditions. This example will demonstrate how to chain multiple functions together - first fetching weather data, then using that information to generate appropriate clothing recommendations. Yes, not very creative, but there is a free API we can use and it should be enough to demonstrate the concept. 

This guide covers: 

1. How does function calling work?
2. When to use function calling?
3. How to define functions for Gemini?
4. How to use function calling with Gemini?
5. Build a weather-based outfit assistant
6. Advanced: Combine structured outputs with function calling
7. Advanced: Function calling with LangChain
8. Advanced: Function calling with OpenAI compatible API

## How does function calling work?

Function calling may imply that the LLM is directly performing some action. This is not the case! When a user prompts an LLM with function calling, the model analyzes the input and determines if and which function would be the most appropriate for the task. Instead of providing a text response, the model might generate a structured JSON object that specifies which function to call and the necessary parameters. 

![Function Intro](../assets/function-intro.png)

In practice function calling not only describe the process of generating structured output, but also the process of calling a function and how to handle the output. As you don't want to return the raw output of the function to your user, you want the LLM to generate an appropriate response, based on the conversation history.

![Function calling](../assets/function-calling.png)


The process follows these steps:
1. Your application sends a prompt to the LLM along with function definitions
2. The LLM analyzes the prompt and decides whether to respond directly or use defined functions
3. If using functions, the LLM generates structured arguments for the function call
4. Your application receives the function call details and executes the actual function
5. The function results are sent back to the LLM
6. The LLM provides a final response incorporating the function results

This cycle can continue as needed, allowing for complex multi-step interactions between the application and the LLM. It is also possible that the LLM decides that it needs to call multiple functions after each other or in parallel. 


For example, when a user asks , our assistant will:
1. Call a weather API to get current conditions in Paris
2. Use that data to generate appropriate clothing recommendations
3. Provide a natural response combining both weather info and outfit suggestions


## When to Use Function Calling?

Function calling has emerged as one of the popular methods for building AI agents. It can help you build human-AI interfaces that access and query real-time information from external sources like APIs, databases, and knowledge bases while providing a natural language interface (text or audio) to the user.

Function calling enables automation tasks like scheduling appointments, creating invoices, or sending reminders. An example usecase could be a customer service assistant might use function calling to seamlessly handle tasks like checking order status, processing returns, and updating customer information – all while maintaining a natural conversation flow with the user.

You now longer need to build Applications which required complex forms or multiple steps to collect information from the user. Instead, you can build a natural language interface that allows the user to interact with the application in a conversational way. Or have no user interface at all and let the LLM interact with the world on your behalf.

## Function Calling with Google Gemini 2.0 Flash

Google Gemini supports function calling through two interfaces, [OpenAPI compatible JSON Schema](https://spec.openapis.org/oas/v3.0.3#schema) and Python functions defintions with docstrings. If you are using JavaScript/Typescript you currently have to use the JSON Schema interface. The Python SDK `google-genai` can automatically generate the JSON Schema from the Python function definitions and docstrings. We are going to take a look at both interfaces. 

_Note: Gemini currently doesn't support `anyOf` type in the JSON Schema._

Lets start with the JSON Schema interface, but before that lets install the `google-genai` library and make sure we have a Gemini API key. If you don't have one yet you can get one from [Google AI Studio](https://aistudio.google.com/app/apikey).

In [ ]:
%pip install "google-genai>=1.0.0" 

Once you have the SDK and API key, you can create a client and define the model you are going to use the new Gemini 2.0 Flash model, which is available via free tier with 1,500 request per day (at 2025-02-06).

In [2]:
import os
from google import genai

# create client
api_key = os.getenv("GEMINI_API_KEY","xxx")
client = genai.Client(api_key=api_key)

# Define the model you are going to use
model_id =  "gemini-2.0-flash"

Before we begin, lets quickly test if we have access to the model and can generate some text.   

In [8]:
res = client.models.generate_content(
    model=model_id,
    contents=["Tell me 1 good fact about Nuremberg."]
)
print(res.text)

Nuremberg is home to the oldest Christmas market in Germany, dating back to the mid-16th century.



### Function Calling with JSON Schema

For using Function Calling with JSON Schema we first need to define our functions using JSON Schema. Let's create a simple weather function that will be the foundation for our outfit assistant. For our schema we need to define:

- `name`: name of the function, this need to match the name of your function in your code 
- `description`: description of what the function does. This is important as this information will be used by the LLM to identify when to use the function
- `parameters`: JSON schema object of type definition for the input arguments of your function. Each parameter has a type, e.g. `string` and a `description` which are used by the LLM what to add here.
- `required`: What `parameters` are required if not all required the LLM might not provide an argument when it thinks its not needed.
 


In [ ]:
weather_function = {
    "name": "get_weather_forecase",
    "description": "Retrieves the weather using Open-Meteo API for a given location (city) and a date (yyy/mm/dd)",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city and state, e.g., San Francisco, CA"
            },
            "date": {
                "type": "string",
                "description": "The date 
            }
        },
        "required": ["location"]
    }
}


For example, when a user asks , our assistant will:
1. Call a weather API to get current conditions in Paris
2. Use that data to generate appropriate clothing recommendations
3. Provide a natural response combining both weather info and outfit suggestions


## Advanced: Combine Structured Outputs with Function Calling

## Advanced: Function Calling with LangChain 

## Advanced: Function Calling with OpenAI integration 